# Basic Monte Carlo Simulation

This notebook demonstrates basic Monte Carlo simulation using the MCMF system.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from monte_carlo_engine.gbm_engine import GeometricBrownianMotionEngine

## Setup Simulation Parameters

In [ ]:
# Simulation parameters
n_simulations = 10000
n_steps = 252  # One year of trading days
initial_price = 100.0
drift = 0.05  # 5% annual drift
volatility = 0.2  # 20% annual volatility

print(f"Simulation Parameters:")
print(f"Number of simulations: {n_simulations:,}")
print(f"Time steps: {n_steps}")
print(f"Initial price: ${initial_price}")
print(f"Annual drift: {drift:.1%}")
print(f"Annual volatility: {volatility:.1%}")

## Create and Run Simulation

In [ ]:
# Create Monte Carlo engine
engine = GeometricBrownianMotionEngine(
    n_simulations=n_simulations,
    n_steps=n_steps,
    initial_price=initial_price,
    drift=drift,
    volatility=volatility,
    random_seed=42  # For reproducible results
)

# Run simulation
print("Running Monte Carlo simulation...")
paths = engine.simulate_paths()
print(f"Simulation complete! Generated {paths.shape} paths with {paths.shape[1]} time points.")

## Analyze Results

In [ ]:
# Extract final prices
final_prices = paths[:, -1]

# Calculate statistics
mean_final = np.mean(final_prices)
std_final = np.std(final_prices)
min_final = np.min(final_prices)
max_final = np.max(final_prices)

# Theoretical expected value
theoretical_mean = initial_price * np.exp(drift)

print(f"\nFinal Price Statistics:")
print(f"Mean: ${mean_final:.2f} (Theoretical: ${theoretical_mean:.2f})")
print(f"Standard Deviation: ${std_final:.2f}")
print(f"Minimum: ${min_final:.2f}")
print(f"Maximum: ${max_final:.2f}")
print(f"\nPercentiles:")
for p in [5, 25, 50, 75, 95]:
    print(f"{p}th percentile: ${np.percentile(final_prices, p):.2f}")

## Visualize Results

In [ ]:
# Create visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Plot sample paths
time_axis = np.arange(n_steps + 1)
for i in range(min(100, n_simulations)):  # Plot first 100 paths
    ax1.plot(time_axis, paths[i], alpha=0.1, color='blue', linewidth=0.5)

# Plot mean path
mean_path = np.mean(paths, axis=0)
ax1.plot(time_axis, mean_path, color='red', linewidth=2, label='Mean Path')
ax1.set_title('Sample Price Paths')
ax1.set_xlabel('Time Steps')
ax1.set_ylabel('Price')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Final price distribution
ax2.hist(final_prices, bins=50, alpha=0.7, color='blue', edgecolor='black')
ax2.axvline(mean_final, color='red', linestyle='--', label=f'Mean: ${mean_final:.2f}')
ax2.axvline(theoretical_mean, color='green', linestyle='--', label=f'Theoretical: ${theoretical_mean:.2f}')
ax2.set_title('Final Price Distribution')
ax2.set_xlabel('Final Price')
ax2.set_ylabel('Frequency')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Returns distribution
returns = np.diff(np.log(paths), axis=1)
all_returns = returns.flatten()
ax3.hist(all_returns, bins=50, alpha=0.7, color='green', edgecolor='black')
ax3.set_title('Daily Returns Distribution')
ax3.set_xlabel('Daily Return')
ax3.set_ylabel('Frequency')
ax3.grid(True, alpha=0.3)

# Q-Q plot
from scipy import stats
stats.probplot(all_returns, dist="norm", plot=ax4)
ax4.set_title('Q-Q Plot vs Normal Distribution')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()